In [ ]:
#@title
%pip install selenium
!apt-get update
!apt install chromium-chromedriver
from selenium import webdriver as wd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import requests
import re
import time
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup as bs
import pandas as pd
import os
from os import path
from datetime import datetime
import sys
import xlwt
import numpy as np

     |████████████████████████████████| 911kB 3.9MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://ppa.launchpad.net/

In [ ]:
chrome_options = wd.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15')
driver = wd.Chrome('chromedriver', chrome_options=chrome_options)


query_url = 'https://www.amazon.com/Apple-Watch-Space-Aluminum-Black/product-reviews/B0799SJ1XX/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews'

driver.get(query_url)
driver.implicitly_wait(10)
res=driver.page_source
obj=bs(res, 'html.parser')
obj


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


<html class=" a-js a-audio a-video a-canvas a-svg a-drag-drop a-geolocation a-history a-webworker a-autofocus a-input-placeholder a-textarea-placeholder a-local-storage a-gradients a-transform3d a-text-shadow a-text-stroke a-box-shadow a-border-radius a-border-image a-opacity a-transform a-transition null" data-19ax5a9jf="dingo" data-aui-build-date="3.21.3-2021-04-08" lang="en-us"><!-- sp:feature:head-start --><head><script async="" crossorigin="anonymous" src="https://c.amazon-adsystem.com/bao-csm/forensics/a9-tq-forensics.min.js"></script><script async="" crossorigin="anonymous" src="https://images-na.ssl-images-amazon.com/images/I/31YXrY93hfL.js"></script><script>var aPageStart = (new Date()).getTime();</script><meta charset="utf-8"/>
<script type="text/javascript">var ue_t0=ue_t0||+new Date();</script>
<!-- sp:feature:cs-optimization -->
<meta content="on" http-equiv="x-dns-prefetch-control"/>
<link href="https://images-na.ssl-images-amazon.com" rel="dns-prefetch"/>
<link href="htt

In [ ]:
rev = obj.find('div', class_ = 'a-row a-spacing-base a-size-base').get_text().strip().split()[4]
rev = int(rev.replace(',', ''))
rev

1330

In [ ]:
#총 리뷰 수 가져오기
#rev = obj.select('#filter-info-section > div > span')[0].get_text().strip()
#rev = int(re.findall('\d+', rev)[1])
#rev = 10000
titles = []
stars = []
dates = []
contents = []
verified_purchase = []
helpful = []
options = []
region_split = []

while len(titles) < rev:
  time.sleep(3)
  source = driver.page_source
  bs_obj = bs(source, 'html.parser')

  #리뷰 타이틀 가져오기
  for i in bs_obj.findAll('a', {'data-hook':'review-title'}):
    titles.append(i.get_text().strip())
  
  #리뷰 날짜 가져오기
  for n in bs_obj.findAll('span', {'data-hook':'review-date'}):
    nn=''.join(n.get_text().split(' ')[-3:])
    date=datetime.strptime(nn, '%B%d,%Y').date()
    dates.append(date)
  
  #리뷰 내용 가져오기
  for a in bs_obj.findAll('span', {'data-hook':'review-body'}):
    contents.append(a.get_text().strip())
  
  #리뷰 평점 가져오기
  for u in bs_obj.findAll('i',{'data-hook':'review-star-rating'}):
    stars.append(int(u.get_text()[0]))
  
  for u in bs_obj.findAll('i', {'data-hook':'cmps-review-star-rating'}):
    stars.append(int(u.get_text()[0]))
  
  #verified_purchase.
  for a in bs_obj.findAll('div', 'a-row a-spacing-mini review-data review-format-strip'):
    verified_purchase.append(a)

  #helpful comments
  for a in bs_obj.findAll('div', class_ = 'a-row a-expander-container a-expander-inline-container cr-vote-action-bar'):
    helpful.append(a)

  #options
  for a in bs_obj.findAll('a', class_ = 'a-size-mini a-link-normal a-color-secondary'):
    options.append(a.get_text())

  #region
  for n in bs_obj.findAll('span', {'data-hook':'review-date'}):
    nn= n.get_text().split(' ')[3:-4]
    region_split.append(nn)

  
  print(' len(titles) = ', len(titles), ' len(helpful) = ', len(helpful), ' len(dates) = ', len(dates),
        ' len(stars) = ', len(stars), ' len(contents) = ', len(contents), ' len(region_split) = ', len(region_split),
        ' len(options) = ', len(options), ' len(verified_purchase = ', len(verified_purchase))
  if len(titles) == rev:
    break
  
  #다음 리뷰 페이지를 넘어갈 수 있는 Next Button 클릭
  
  driver.find_element_by_xpath('//*[@id="cm_cr-pagination_bar"]/ul/li[2]/a').click()
  WebDriverWait(driver, 15).until(
    # 지정한 한개 요소가 올라면 웨이트 종료?
      EC.presence_of_element_located((By.XPATH, '//*[@id="a-autoid-3"]/span/input'))
  )


driver.close()
driver.quit()

#df= pd.DataFrame({'Date': dates, 'Rating': stars, 'Title': titles, 'Body': contents})

#df

 len(titles) =  10  len(helpful) =  10  len(dates) =  10  len(stars) =  10  len(contents) =  10  len(region_split) =  10  len(options) =  4  len(verified_purchase =  10
 len(titles) =  20  len(helpful) =  20  len(dates) =  20  len(stars) =  20  len(contents) =  20  len(region_split) =  20  len(options) =  8  len(verified_purchase =  20
 len(titles) =  30  len(helpful) =  30  len(dates) =  30  len(stars) =  30  len(contents) =  30  len(region_split) =  30  len(options) =  17  len(verified_purchase =  30
 len(titles) =  40  len(helpful) =  40  len(dates) =  40  len(stars) =  40  len(contents) =  40  len(region_split) =  40  len(options) =  26  len(verified_purchase =  40
 len(titles) =  50  len(helpful) =  50  len(dates) =  50  len(stars) =  50  len(contents) =  50  len(region_split) =  50  len(options) =  34  len(verified_purchase =  50
 len(titles) =  60  len(helpful) =  60  len(dates) =  60  len(stars) =  60  len(contents) =  60  len(region_split) =  60  len(options) =  43  len(verifi

NoSuchElementException: ignored

In [ ]:
 print(' len(titles) = ', len(titles), ' len(helpful) = ', len(helpful), ' len(dates) = ', len(dates),
        ' len(stars) = ', len(stars), ' len(contents) = ', len(contents), ' len(region_split) = ', len(region_split),
        ' len(options) = ', len(options))

 len(titles) =  5000  len(helpful) =  5000  len(dates) =  5000  len(stars) =  5000  len(contents) =  5000  len(region_split) =  5000  len(options) =  5000


In [ ]:
dates = dates[0: 1260]
titles = titles[0: 1260]
helpful = helpful[0:1260]
stars = stars[0:1260]
contents = contents[0:1260]
region_split = region_split[0: 1260]
options = options[0:1260]
verified_purchase = verified_purchase[0:1260]

In [ ]:
df= pd.DataFrame({'Date': dates, 'Rating': stars, 'Verified_Purchase': verified_purchase, 
                  'Helpful': helpful, 'Options' : options,'Region_Split' : region_split,
                  'Title': titles, 'Body': contents})

df

,Date,Rating,Verified_Purchase,Helpful,Region_Split,Title,Body
0,2019-01-02,1,"[[Color: Space Gray Aluminum Case, [], Size: 4...","[[\n, [<span class=""a-size-base a-color-tertia...","[United, States]",Buyers beware!!!,Let me start with I love amazon but wherever t...
1,2018-05-13,1,"[[[<span class=""a-size-mini a-color-state a-te...","[[\n, [<span class=""a-size-base a-color-tertia...","[United, States]",I Got An Empty Box,I wish there was a zero star option. Ordered t...
2,2018-10-24,5,"[[Color: Gold Aluminum Case, [], Size: 38 mm, ...","[[\n, [<span class=""a-size-base a-color-tertia...","[United, States]",5 Stars 😍,I was skeptical of buying a refurbished produc...
3,2018-03-15,5,"[[[<span class=""a-size-mini a-color-state a-te...","[[\n, [<span class=""a-size-base a-color-tertia...","[United, States]",Great watch to keep me active and connected!,This is the most expensive watch I have ever p...
4,2018-04-25,5,"[[[<span class=""a-size-mini a-color-state a-te...","[[\n, [<span class=""a-size-base a-color-tertia...","[United, States]",Excellent watch,This is my first apple watch purchase. Wish I...
...,...,...,...,...,...,...,...
1255,2021-04-07,5,"[[Color: Space Gray Aluminum Case, [], Size: 4...","[[\n, [\n, <span class=""a-button a-button-base...","[United, States]",Great buy,Works and looks great
1256,2021-04-03,5,"[[Color: Silver Aluminum Case, [], Size: 38 mm...","[[\n, [<span class=""a-size-base a-color-tertia...","[United, States]",Love it,Connected good! good product
1257,2021-02-22,5,"[[Color: Space Gray Aluminum Case, [], Size: 3...","[[\n, [<span class=""a-size-base a-color-tertia...","[United, States]",Reemplazo de Apple Watch,Your browser does not support HTML5 video.\n\n...
1258,2021-03-09,5,"[[Color: Space Gray Aluminum Case, [], Size: 4...","[[\n, [\n, <span class=""a-button a-button-base...","[United, States]",Provides all the essentials,Bought this along with a case. I was tempted t...


In [ ]:
df['Options'] = None

In [ ]:
#helpful

for i in range(len(df)):
  df['Helpful'][i] = df['Helpful'][i].get_text().split('\n')[1]

for i in range(len(df)):
  try:
    df['Helpful'][i] = df['Helpful'][i].split()[0]
    df['Helpful'] = df['Helpful'].str.replace('One', '1')
    df['Helpful'] = df['Helpful'].str.replace(',', '').apply(pd.to_numeric)
  except:
    pass

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
#verified_purchase
for i in range(len(df['Verified_Purchase'])):
  df['Verified_Purchase'][i] = df['Verified_Purchase'][i].get_text()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
#region
region = []
df['Region'] = df['Region_Split']
for j in range(len(df['Region_Split'])):
  df['Region'][j] = ''
  for i in range(len(df['Region_Split'][j])):
    df['Region'][j] = df['Region'][j] + ' ' + df['Region_Split'][j][i]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [ ]:
df = df.drop(columns = ['Region_Split'])
df

,Date,Rating,Verified_Purchase,Helpful,Title,Body,Options,Region
0,2019-01-02,1,Color: Space Gray Aluminum CaseSize: 42 mmStyl...,527.0,Buyers beware!!!,Let me start with I love amazon but wherever t...,None,United States
1,2018-05-13,1,Verified Purchase,303.0,I Got An Empty Box,I wish there was a zero star option. Ordered t...,None,United States
2,2018-10-24,5,Color: Gold Aluminum CaseSize: 38 mmStyle: Pin...,125.0,5 Stars 😍,I was skeptical of buying a refurbished produc...,None,United States
3,2018-03-15,5,Verified Purchase,100.0,Great watch to keep me active and connected!,This is the most expensive watch I have ever p...,None,United States
4,2018-04-25,5,Verified Purchase,94.0,Excellent watch,This is my first apple watch purchase. Wish I...,None,United States
...,...,...,...,...,...,...,...,...
1255,2021-04-07,5,Color: Space Gray Aluminum CaseSize: 42 mmStyl...,NaN,Great buy,Works and looks great,None,United States
1256,2021-04-03,5,Color: Silver Aluminum CaseSize: 38 mmStyle: W...,1.0,Love it,Connected good! good product,None,United States
1257,2021-02-22,5,Color: Space Gray Aluminum CaseSize: 38 mmStyl...,1.0,Reemplazo de Apple Watch,Your browser does not support HTML5 video.\n\n...,None,United States
1258,2021-03-09,5,Color: Space Gray Aluminum CaseSize: 42 mmStyl...,NaN,Provides all the essentials,Bought this along with a case. I was tempted t...,None,United States


In [ ]:
df['Product_Name'] = 'Apple Watch Series 3 (GPS, 42MM) - Space Gray Aluminum Case with Black Sport Band (Renewed)'
df

,Date,Rating,Verified_Purchase,Helpful,Title,Body,Options,Region,Product_Name
0,2019-01-02,1,Color: Space Gray Aluminum CaseSize: 42 mmStyl...,527.0,Buyers beware!!!,Let me start with I love amazon but wherever t...,None,United States,"Apple Watch Series 3 (GPS, 42MM) - Space Gray ..."
1,2018-05-13,1,Verified Purchase,303.0,I Got An Empty Box,I wish there was a zero star option. Ordered t...,None,United States,"Apple Watch Series 3 (GPS, 42MM) - Space Gray ..."
2,2018-10-24,5,Color: Gold Aluminum CaseSize: 38 mmStyle: Pin...,125.0,5 Stars 😍,I was skeptical of buying a refurbished produc...,None,United States,"Apple Watch Series 3 (GPS, 42MM) - Space Gray ..."
3,2018-03-15,5,Verified Purchase,100.0,Great watch to keep me active and connected!,This is the most expensive watch I have ever p...,None,United States,"Apple Watch Series 3 (GPS, 42MM) - Space Gray ..."
4,2018-04-25,5,Verified Purchase,94.0,Excellent watch,This is my first apple watch purchase. Wish I...,None,United States,"Apple Watch Series 3 (GPS, 42MM) - Space Gray ..."
...,...,...,...,...,...,...,...,...,...
1255,2021-04-07,5,Color: Space Gray Aluminum CaseSize: 42 mmStyl...,NaN,Great buy,Works and looks great,None,United States,"Apple Watch Series 3 (GPS, 42MM) - Space Gray ..."
1256,2021-04-03,5,Color: Silver Aluminum CaseSize: 38 mmStyle: W...,1.0,Love it,Connected good! good product,None,United States,"Apple Watch Series 3 (GPS, 42MM) - Space Gray ..."
1257,2021-02-22,5,Color: Space Gray Aluminum CaseSize: 38 mmStyl...,1.0,Reemplazo de Apple Watch,Your browser does not support HTML5 video.\n\n...,None,United States,"Apple Watch Series 3 (GPS, 42MM) - Space Gray ..."
1258,2021-03-09,5,Color: Space Gray Aluminum CaseSize: 42 mmStyl...,NaN,Provides all the essentials,Bought this along with a case. I was tempted t...,None,United States,"Apple Watch Series 3 (GPS, 42MM) - Space Gray ..."


In [ ]:
df['Link'] = 'https://www.amazon.com/Apple-Watch-Space-Aluminum-Black/product-reviews/B0799SJ1XX/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews'

In [ ]:
df = df[['Product_Name', 'Date', 'Rating', 'Verified_Purchase',	'Helpful',	'Options',	'Region', 'Title',	'Body', 'Link']]
df

,Product_Name,Date,Rating,Verified_Purchase,Helpful,Options,Region,Title,Body,Link
0,"Apple Watch Series 3 (GPS, 42MM) - Space Gray ...",2019-01-02,1,Color: Space Gray Aluminum CaseSize: 42 mmStyl...,527.0,None,United States,Buyers beware!!!,Let me start with I love amazon but wherever t...,https://www.amazon.com/Apple-Watch-Space-Alumi...
1,"Apple Watch Series 3 (GPS, 42MM) - Space Gray ...",2018-05-13,1,Verified Purchase,303.0,None,United States,I Got An Empty Box,I wish there was a zero star option. Ordered t...,https://www.amazon.com/Apple-Watch-Space-Alumi...
2,"Apple Watch Series 3 (GPS, 42MM) - Space Gray ...",2018-10-24,5,Color: Gold Aluminum CaseSize: 38 mmStyle: Pin...,125.0,None,United States,5 Stars 😍,I was skeptical of buying a refurbished produc...,https://www.amazon.com/Apple-Watch-Space-Alumi...
3,"Apple Watch Series 3 (GPS, 42MM) - Space Gray ...",2018-03-15,5,Verified Purchase,100.0,None,United States,Great watch to keep me active and connected!,This is the most expensive watch I have ever p...,https://www.amazon.com/Apple-Watch-Space-Alumi...
4,"Apple Watch Series 3 (GPS, 42MM) - Space Gray ...",2018-04-25,5,Verified Purchase,94.0,None,United States,Excellent watch,This is my first apple watch purchase. Wish I...,https://www.amazon.com/Apple-Watch-Space-Alumi...
...,...,...,...,...,...,...,...,...,...,...
1255,"Apple Watch Series 3 (GPS, 42MM) - Space Gray ...",2021-04-07,5,Color: Space Gray Aluminum CaseSize: 42 mmStyl...,NaN,None,United States,Great buy,Works and looks great,https://www.amazon.com/Apple-Watch-Space-Alumi...
1256,"Apple Watch Series 3 (GPS, 42MM) - Space Gray ...",2021-04-03,5,Color: Silver Aluminum CaseSize: 38 mmStyle: W...,1.0,None,United States,Love it,Connected good! good product,https://www.amazon.com/Apple-Watch-Space-Alumi...
1257,"Apple Watch Series 3 (GPS, 42MM) - Space Gray ...",2021-02-22,5,Color: Space Gray Aluminum CaseSize: 38 mmStyl...,1.0,None,United States,Reemplazo de Apple Watch,Your browser does not support HTML5 video.\n\n...,https://www.amazon.com/Apple-Watch-Space-Alumi...
1258,"Apple Watch Series 3 (GPS, 42MM) - Space Gray ...",2021-03-09,5,Color: Space Gray Aluminum CaseSize: 42 mmStyl...,NaN,None,United States,Provides all the essentials,Bought this along with a case. I was tempted t...,https://www.amazon.com/Apple-Watch-Space-Alumi...


In [ ]:
df.to_csv('Apple Watch Series 3 (GPS, 42MM) - Space Gray Aluminum Case with Black Sport Band (Renewed).csv', index = 0)